In [1]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping



In [2]:

# Check available GPUs
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

# Enable multi-GPU training
strategy = tf.distribute.MirroredStrategy()



Num GPUs Available: 2


In [3]:
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
with strategy.scope():
    # -----------------------------
    # Data Loading and Preprocessing
    # -----------------------------
    df = pd.read_csv('/kaggle/input/churnprediction/Churn_Modelling.csv')
    df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
    
    # One-hot encoding for Geography
    df = pd.get_dummies(df, columns=['Geography'], drop_first=True)
    
    # Label encoding for Gender
    le = LabelEncoder()
    df['Gender'] = le.fit_transform(df['Gender'])
    
    # Separate features and target
    X = df.drop('Exited', axis=1)
    y = df['Exited']
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # -----------------------------
    # Building the Multi-GPU Neural Network Model with LeakyReLU
    # -----------------------------
    model = Sequential()
    
    # Input Layer
    model.add(Dense(128, input_dim=X_train.shape[1]))
    model.add(LeakyReLU(alpha=0.01))  # LeakyReLU activation instead of 'relu'
    model.add(Dropout(0.2))
    
    # Hidden Layer 1
    model.add(Dense(64))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.2))
    
    # Hidden Layer 2
    model.add(Dense(32))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.2))
    
    # Output Layer
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model with multi-GPU support
    model.compile(optimizer='adamW', loss='binary_crossentropy', metrics=['accuracy'])

# -----------------------------
# Training the Model
# -----------------------------
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stop])

# -----------------------------
# Evaluating the Model
# -----------------------------
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Accuracy:', accuracy)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.7530 - loss: 0.5216 - val_accuracy: 0.8413 - val_loss: 0.3832
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8328 - loss: 0.4119 - val_accuracy: 0.8612 - val_loss: 0.3482
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8486 - loss: 0.3762 - val_accuracy: 0.8562 - val_loss: 0.3431
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8490 - loss: 0.3679 - val_accuracy: 0.8637 - val_loss: 0.3354
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8479 - loss: 0.3650 - val_accuracy: 0.8637 - val_loss: 0.3307
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8612 - loss: 0.3474 - val_accuracy: 0.8662 - val_loss: 0.3341
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8650 - loss: 0.3449 - val_accuracy: 0.8650 - val_loss: 0.3338
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8597 - loss: 0.3427 - val_acc